In [ ]:
#playing around with multisets to spread out weight of each profile over events/nodes


In [7]:
import datasketch 
import numpy as np
import time
import ete3


In [8]:
from datasketch import WeightedMinHashGenerator

v1 = 1000 *[.5]
v2 = 1000 * [1]

# WeightedMinHashGenerator requires dimension as the first argument
wmg = WeightedMinHashGenerator(1000, sample_size=512, seed=0)

wm1 = wmg.minhash(v1) # wm1 is of the type WeightedMinHash
wm2 = wmg.minhash(v2)

print("Estimated Jaccard is", wm1.jaccard(wm2))



('Estimated Jaccard is', 0.482421875)


In [9]:
#each hog has 4000 taxonomic elements
#total weight is defined as 1 

#spread out the hog over its taxonomic range 
#give more weight to certain events/levels

#score over the HOG -> normalize

import numpy as np




In [11]:
mastertree = ete3.Tree( 'mastertree.nwk' , format = 1 )  
print(len(mastertree))
treeindex = { n.name : i for i,n in enumerate(mastertree.traverse())}
print(len(treeindex))

2103
2997


NameError: name 'descendants' is not defined

In [13]:
#assign a weight to each taxonomic level. decrese exponentially or linearly?
#exp should be tuned for each event type

#todo sep functions for 3 kingdoms... 
exp_bact = .9
exp_euk = .9
exp_arch = .9


for node in mastertree.traverse():
    node.add_feature('degree', 1 )
for node in mastertree.iter_descendants():
    for d in node.iter_descendants():
        d.degree+=1
degreedict = {n.name : n.degree for n in mastertree.traverse()}        
print(degreedict)

{'': 1, '477974': 7, '2065118': 8, '65093': 7, '460265': 6, '429572': 8, '13735': 15, '89373': 6, '374930': 7, '374931': 7, '77166': 15, '507522': 8, '929556': 6, '929557': 6, '351348': 7, '161': 7, '443149': 10, '340': 7, '407976': 7, '443143': 7, '953739': 7, '32524': 12, '32523': 11, '443144': 7, '680646': 8, '7375': 17, '354276': 7, '340102': 8, '340100': 7, '765698': 7, '214092': 9, '652620': 8, '759364': 6, '1198451': 5, '869209': 6, '759362': 7, '759363': 6, '1093141': 3, '4734': 8, '1054460': 8, '57723': 2, '441772': 10, '441771': 10, '441770': 10, '6296': 10, '294': 8, '1005048': 6, '4641': 9, '663917': 8, '1643826': 5, '1643824': 6, '63737': 6, '448385': 6, '43988': 6, '32011': 5, '274': 5, '868131': 6, '670386': 4, '148305': 12, '766764': 8, '998088': 6, '130081': 4, '1201294': 7, '315277': 6, '471854': 7, '349519': 8, '665079': 12, '552526': 11, '2263': 4, '2260': 4, '2266': 5, '2267': 6, '357804': 5, '485917': 6, '485916': 8, '485915': 6, '485914': 6, '334390': 9, '9526': 

In [7]:
import pyham

ortho = ''
treestr = mastertree.write(format = 1)




In [ ]:

#add an exponential bleed function for events gain, dup, loss
#bleedup, bleeddown should be diff for each event type

bleedup = .1
bleeddown = .1
#assign score to nodes

#traverse tree profile and calculate scores
for n in treeprofile.traverse():
    #add score for an event based on degree
    n.add_feature('score', exp_bact**degreedict[n.name])

#blur scores up and down
for e in treeprofile.traverse():
    if e.score>0:
        init = e.score
        for n in e.iter_ancestors():
            if n.score:
                n.score *= bleedup
            else:
                n.score = bleedup

        for n in e.iter_descendants():
            if n.score:
                n.score *= bleeddown
            else:
                n.score = bleeddown

#after a treeprofile is distributed to it taxonomic nodes, normalize
vector = np.asarray([ n.score for n in treeprofile.traverse() ]) 
vector /= np.sum(vector)
#turn into weighted minhash

